In [2]:
import os
path='/home/ubuntu/workspace/t1/'
pypath='/home/ubuntu/workspace/t1/PythonCode'
os.chdir(path)
import django
django.setup()
os.chdir(pypath)


from openpyxl import load_workbook, Workbook, cell
import pandas as pd, numpy as np
from FileProcessingFunctions import ifHeaderReturnIndex, isRowValid
from django.shortcuts import render
from django.contrib.auth.decorators import login_required
from django.http import HttpResponse
from prof.forms import FileSpaceForm
from prof.models import FileSpace, RunSpace, TransactionData, PnLData
from django.http import HttpResponseRedirect
from django.core.urlresolvers import reverse
from guardian.shortcuts import get_objects_for_user, assign_perm, get_perms
from django.contrib.auth.models import User
from guardian.decorators import permission_required_or_403
from django.db import transaction



def createKeyTemplate(transactionfile):
    Tr = pd.read_excel(transactionfile)
    Tr['Month']=Tr['Date'].apply(lambda x: str(x.year)+'-'+ ('0'+str(x.month))[-2:])
    KeyTemplate=Tr[['Business Unit','Product Number','Month', 'List Price']].groupby( ['Business Unit','Product Number','Month']).sum().round().unstack()
    KeyTemplate.columns=KeyTemplate.columns.droplevel(0)
    KeyTemplate.columns.name=None
    KeyTemplate[KeyTemplate!=np.nan]=None
    KeyTemplate=KeyTemplate.reset_index()
    KeyTemplate[lastColumnName] = np.nan
    return KeyTemplate
    
    
    
def createQuantityKey(transactionfile):
    Tr = pd.read_excel(transactionfile)
    Tr['Month']=Tr['Date'].apply(lambda x: str(x.year)+'-'+ ('0'+str(x.month))[-2:])
    QKey=Tr[['Business Unit','Product Number','Month', 'Quantity']].groupby( ['Business Unit','Product Number','Month']).sum().round().unstack()
    QKey.columns=QKey.columns.droplevel(0)
    QKey.columns.name=None
    #QKey[QKey!=np.nan]=None
    #QKey['Trailing Twelve Months'] = np.nan
    QKeyAgg=Tr[['Business Unit','Product Number', 'Quantity']].groupby( ['Business Unit','Product Number']).sum().round()
    QKeyAgg.rename(columns={'Quantity': lastColumnName},inplace=True)
    QKey=pd.concat([QKey,QKeyAgg],axis=1)
    QKey=QKey.reset_index().fillna(0)
    return QKey

In [10]:
run=RunSpace.objects.get(pk=1)
